# ClusterTree

A `ClusterTree` is a more general structure than an `EliminationTree` or `JunctionTree`. It represents a tree where each node (a 'Cluster') contains a subset of factors from an original factor graph. The key property is that the tree must be 'family preserving', meaning each original factor must belong entirely within a single cluster.

`ClusterTree` itself is a base class. `EliminatableClusterTree` adds the ability to perform elimination, and `JunctionTree` is a specific type of `EliminatableClusterTree` derived from an `EliminationTree`.

Direct use of `ClusterTree` in typical Python applications is less common than `JunctionTree` or `BayesTree`, as it's often an intermediate representation.

<a href="https://colab.research.google.com/github/borglab/gtsam/blob/develop/gtsam/inference/doc/ClusterTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install gtsam

In [7]:
import gtsam
import numpy as np

# Note: ClusterTree itself might not be directly exposed or used.
# We typically interact with JunctionTree or BayesTree which build upon it.
# We'll demonstrate concepts using JunctionTree which inherits Cluster features.
from gtsam import GaussianFactorGraph, Ordering, VariableIndex, GaussianBayesTree
from gtsam import symbol_shorthand

X = symbol_shorthand.X
L = symbol_shorthand.L

## Concept and Relation to JunctionTree

A `JunctionTree` *is a* `ClusterTree` (specifically, an `EliminatableClusterTree`). It's constructed during multifrontal elimination. Each node in the `JunctionTree` is a `Cluster` containing factors that will be eliminated together to form a clique in the resulting `BayesTree`.

We will create a `JunctionTree` and examine its properties, which include those inherited from `ClusterTree`.

In [8]:
# Create a graph (same as BayesTree example)
graph = GaussianFactorGraph()
model = gtsam.noiseModel.Isotropic.Sigma(1, 1.0)
graph.add(X(0), -np.eye(1), np.zeros(1), model)
graph.add(X(0), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(X(1), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(0), np.eye(1), np.zeros(1), model)
graph.add(L(1), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(1), np.eye(1), np.zeros(1), model)
graph.add(L(2), -np.eye(1), X(2), np.eye(1), np.zeros(1), model)

ordering = Ordering.Colamd(VariableIndex(graph))

# Create a Junction Tree (implicitly uses ClusterTree structure)
# Note: JunctionTree constructor might not be directly exposed.
# It's usually an intermediate in eliminateMultifrontal.
# We might need to construct it indirectly or focus on BayesTree access.

# Let's get the BayesTree first, as JunctionTree creation is internal.
bayes_tree = graph.eliminateMultifrontal(ordering)

# We can print the BayesTree, which shows the cluster structure
# (Cliques in BayesTree correspond to Clusters in JunctionTree)
print("Junction Tree (as ClusterTree): ")
bayes_tree.print() # Printing BayesTree shows clique structure

# Access root cluster(s)
roots = bayes_tree.roots()
if roots:
  print("\nAccessing a root cluster (node):")
  root_clique = roots[0]
  # In the JunctionTree, this node would contain the factors that *produced*
  # the conditional in the BayesTree clique. We can see the involved keys.
  root_clique.print("", gtsam.DefaultKeyFormatter) # Print clique details

print(f"\nNumber of roots: {len(roots)}")

# Direct instantiation or manipulation of Cluster/JunctionTree nodes
# is less common in Python than using the results (BayesTree).

Junction Tree (as ClusterTree): 
: cliques: 2, variables: 5
- p(x1 l2 x2 )
  R = [   1.61245 -0.620174 -0.620174 ]
      [         0   1.27098  -1.08941 ]
      [         0         0  0.654654 ]
  d = [ 0 0 0 ]
  mean: 3 elements
  l2: 0
  x1: 0
  x2: 0
  logNormalizationConstant: -2.46292
  No noise model
| - p(l1 x0  | x1)
  R = [   1.41421 -0.707107 ]
      [         0   1.58114 ]
  S[x1] = [ -0.707107 ]
          [ -0.948683 ]
  d = [ 0 0 ]
  logNormalizationConstant: -1.03316
  No noise model


AttributeError: 'gtsam.gtsam.GaussianBayesTree' object has no attribute 'roots'